# A quick look at Pandas GroupBy

In [1]:
import numpy as np
import pandas as pd

Let's make a toy DF (example inspired by Wes McKinney's [Python for Data Analysis](http://proquest.safaribooksonline.com.libproxy.berkeley.edu/book/programming/python/9781491957653):

In [23]:
df = pd.read_csv("elections.csv")
df.head()

Candidate        Party     %  Year Result
0    Reagan   Republican  50.7  1980    win
1    Carter   Democratic  41.0  1980   loss
2  Anderson  Independent   6.6  1980   loss
3    Reagan   Republican  58.8  1984    win
4   Mondale   Democratic  37.6  1984   loss

Let's group the `%` column by the `Party` column. A call to [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) does that, but what is the object that results?

In [115]:
percent_grouped_by_party = df['%'].groupby(df['Party'])
percent_grouped_by_party.groups

{'Democratic': Int64Index([1, 4, 6, 7, 10, 13, 15, 17, 19, 21], dtype='int64'),
 'Independent': Int64Index([2, 9, 12], dtype='int64'),
 'Republican': Int64Index([0, 3, 5, 8, 11, 14, 16, 18, 20, 22], dtype='int64')}

As we see, `percent_grouped_by_party` is **NOT** a new DataFrame. Instead, it's a `SeriesGroupBy` object. A `SeriesGroupBy` consists of `groups`, one for each of the distinct values of the `Party` column. If we ask to see these groups, we'll be able to see which indices in the original DataFrame correspond to each group.

In [31]:
percent_grouped_by_party.groups

{'Democratic': Int64Index([1, 4, 6, 7, 10, 13, 15, 17, 19, 21], dtype='int64'),
 'Independent': Int64Index([2, 9, 12], dtype='int64'),
 'Republican': Int64Index([0, 3, 5, 8, 11, 14, 16, 18, 20, 22], dtype='int64')}

The `percent_grouped_by_party` object is capable of making computations across all these groups. For example, if we call the `mean` function, we'll get the mean of the "Democratic" `Series`, the mean of the "Independent" `Series`, and the mean of the "Republican" `Series`.

In [32]:
percent_grouped_by_party.mean()

Party
Democratic     46.53
Independent    11.30
Republican     47.86
Name: %, dtype: float64

You'll pretty much never do this when working with real data, but we can iterate over a `groupby` object. As we iterate we get pairs of `(name, group)`, where `name` is a String label for the group, and `group` is a `Series` corresponding to the all the values from the given group.

In [40]:
from IPython.display import display  # like print, but for complex objects

for name, group in grouped:
    print('Name:', name)
    print(type(group))
    display(group)

Name: Democratic
<class 'pandas.core.series.Series'>


1     41.0
4     37.6
6     45.6
7     43.0
10    49.2
13    48.4
15    48.3
17    52.9
19    51.1
21    48.2
Name: %, dtype: float64

Name: Independent
<class 'pandas.core.series.Series'>


2      6.6
9     18.9
12     8.4
Name: %, dtype: float64

Name: Republican
<class 'pandas.core.series.Series'>


0     50.7
3     58.8
5     53.4
8     37.4
11    40.7
14    47.9
16    50.7
18    45.7
20    47.2
22    46.1
Name: %, dtype: float64

We can also group by multiple columns. For example, suppose we want to track all combinations of `{'Democratic', 'Republican', and 'Independent'}` and `{'win', 'loss'}`. 

In [114]:
g2 = df['%'].groupby([df['Party'], df['Year']])
g2.groups

{('Democratic', 1980): Int64Index([1], dtype='int64'),
 ('Democratic', 1984): Int64Index([4], dtype='int64'),
 ('Democratic', 1988): Int64Index([6], dtype='int64'),
 ('Democratic', 1992): Int64Index([7], dtype='int64'),
 ('Democratic', 1996): Int64Index([10], dtype='int64'),
 ('Democratic', 2000): Int64Index([13], dtype='int64'),
 ('Democratic', 2004): Int64Index([15], dtype='int64'),
 ('Democratic', 2008): Int64Index([17], dtype='int64'),
 ('Democratic', 2012): Int64Index([19], dtype='int64'),
 ('Democratic', 2016): Int64Index([21], dtype='int64'),
 ('Independent', 1980): Int64Index([2], dtype='int64'),
 ('Independent', 1992): Int64Index([9], dtype='int64'),
 ('Independent', 1996): Int64Index([12], dtype='int64'),
 ('Republican', 1980): Int64Index([0], dtype='int64'),
 ('Republican', 1984): Int64Index([3], dtype='int64'),
 ('Republican', 1988): Int64Index([5], dtype='int64'),
 ('Republican', 1992): Int64Index([8], dtype='int64'),
 ('Republican', 1996): Int64Index([11], dtype='int64'),

Given this groupby object, we can compute the average percentage earned every time each of the parties won and lost the presidential election. We see that at least between 1980 and 2016, the Republicans have typically lost and won their elections by wider margins.

In [33]:
g2.mean()

Party        Result
Democratic   loss      44.850000
             win       49.050000
Independent  loss      11.300000
Republican   loss      42.750000
             win       51.266667
Name: %, dtype: float64

## DataFrameGroupBy

We can also group an entire dataframe by one or more columns. This results in a `DataFrameGroupBy` object as the result:

In [34]:
everything_grouped_by_party = df.groupby('Party')
everything_grouped_by_party

As in our previous example, this object contains three `group` objects, one for each party label.

In [35]:
everything_grouped_by_party.groups

{'Democratic': Int64Index([1, 4, 6, 7, 10, 13, 15, 17, 19, 21], dtype='int64'),
 'Independent': Int64Index([2, 9, 12], dtype='int64'),
 'Republican': Int64Index([0, 3, 5, 8, 11, 14, 16, 18, 20, 22], dtype='int64')}

Just as with `SeriesGroupBy` objects, we can iterate over a `DataFrameGroupBy` object to understand what is effectively inside.

In [20]:
for n, g in k1g:
    print('name:', n)
    display(g)

name: Democratic


Candidate       Party     %  Year Result
1     Carter  Democratic  41.0  1980   loss
4    Mondale  Democratic  37.6  1984   loss
6    Dukakis  Democratic  45.6  1988   loss
7    Clinton  Democratic  43.0  1992    win
10   Clinton  Democratic  49.2  1996    win
13      Gore  Democratic  48.4  2000   loss
15     Kerry  Democratic  48.3  2004   loss
17     Obama  Democratic  52.9  2008    win
19     Obama  Democratic  51.1  2012    win
21   Clinton  Democratic  48.2  2016   loss

name: Independent


Candidate        Party     %  Year Result
2   Anderson  Independent   6.6  1980   loss
9      Perot  Independent  18.9  1992   loss
12     Perot  Independent   8.4  1996   loss

name: Republican


Candidate       Party     %  Year Result
0     Reagan  Republican  50.7  1980    win
3     Reagan  Republican  58.8  1984    win
5       Bush  Republican  53.4  1988    win
8       Bush  Republican  37.4  1992   loss
11      Dole  Republican  40.7  1996   loss
14      Bush  Republican  47.9  2000    win
16      Bush  Republican  50.7  2004    win
18    McCain  Republican  45.7  2008   loss
20    Romney  Republican  47.2  2012   loss
22     Trump  Republican  46.1  2016    win

And just like `SeriesGroupBy` objects, we can apply functions like `mean` to compute the mean of each group. Since a `DataFrameGroupBy` is linked to the entire original dataframe (instead of to a single column from the dataframe), we calculate a mean for every numerical column. In this example below, we get the mean vote earned (as before), and the mean year (which isn't a useful quantity).

In [94]:
everything_grouped_by_party.mean()

%         Year
Party                          
Democratic   46.53  1998.000000
Independent  11.30  1989.333333
Republican   47.86  1998.000000

Where did all the other columns go in the mean above? They are *nuisance columns*, which get automatically eliminated from an operation where it doesn't make sense (such as a numerical mean).

Both `SeriesGroupBy` and `DataFrameGroupBy` objects have lots of handy functions for computing aggregate values for groups. A few are demoed below.

In [95]:
everything_grouped_by_party.min()

Candidate     %  Year Result
Party                                   
Democratic     Carter  37.6  1980   loss
Independent  Anderson   6.6  1980   loss
Republican       Bush  37.4  1980   loss

In [96]:
everything_grouped_by_party.size()

Party
Democratic     10
Independent     3
Republican     10
dtype: int64

We can even define our own custom aggregation functions. For example, the function below returns the first item in a series.

In [101]:
def average_of_first_and_last(series):
    return (series.iloc[0] + series.iloc[-1])/2

We can supply this function as a custom aggregation function for each series. As you can see, nuisance columns are automatically removed.

In [103]:
everything_grouped_by_party.agg(average_of_first_and_last)

%  Year
Party                  
Democratic   44.6  1998
Independent   7.5  1988
Republican   48.4  1998

## Grouping over a different dimension (extra)

Above, we've been grouping data along the rows, using column keys as our selectors.  But we can also group along the columns, for example we can group by data type:

In [21]:
df.dtypes

Candidate     object
Party         object
%            float64
Year           int64
Result        object
dtype: object

In [22]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    display(group)

int64


Year
0   1980
1   1980
2   1980
3   1984
4   1984
5   1988
6   1988
7   1992
8   1992
9   1992
10  1996
11  1996
12  1996
13  2000
14  2000
15  2004
16  2004
17  2008
18  2008
19  2012
20  2012
21  2016
22  2016

float64


%
0   50.7
1   41.0
2    6.6
3   58.8
4   37.6
5   53.4
6   45.6
7   43.0
8   37.4
9   18.9
10  49.2
11  40.7
12   8.4
13  48.4
14  47.9
15  48.3
16  50.7
17  52.9
18  45.7
19  51.1
20  47.2
21  48.2
22  46.1

object


Candidate        Party Result
0     Reagan   Republican    win
1     Carter   Democratic   loss
2   Anderson  Independent   loss
3     Reagan   Republican    win
4    Mondale   Democratic   loss
5       Bush   Republican    win
6    Dukakis   Democratic   loss
7    Clinton   Democratic    win
8       Bush   Republican   loss
9      Perot  Independent   loss
10   Clinton   Democratic    win
11      Dole   Republican   loss
12     Perot  Independent   loss
13      Gore   Democratic   loss
14      Bush   Republican    win
15     Kerry   Democratic   loss
16      Bush   Republican    win
17     Obama   Democratic    win
18    McCain   Republican   loss
19     Obama   Democratic    win
20    Romney   Republican   loss
21   Clinton   Democratic   loss
22     Trump   Republican    win